In [1]:
import os
import json
import numpy as np
import io
from datetime import datetime
from data_generator import DataGenerator
from trade_env import TraderEnv
from rnn.model_keras import NeuralNetwork
import numpy
import matplotlib.pyplot as plt
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense, Activation, LSTM
from sklearn.preprocessing import MinMaxScaler

/Users/samirantonio/anaconda/envs/python3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
dt = DataGenerator(random=False, first_index=10)
dt.rewind()

In [3]:
# integer encode input data
def onehot_encoded (integer_encoded, char_to_int = 3):
    # one hot encode
    onehot_encoded = list()
    letter = [0 for _ in range(char_to_int)]
    letter[integer_encoded] = 1
    onehot_encoded.append(letter)
    
    return onehot_encoded[0]

In [4]:
last_price = 0
last_time = 0
steps = 100
def get_state(raw_state):
    global last_price
    global last_time
    list = []

    price = raw_state["price"]

    def prepare_orders(orders, price, multi):
        for order in orders:
            list.append((float(order[0])/price) * multi)
            #list.append(float(order[1]))

    bids = raw_state["bids"][:5]
    asks = raw_state["asks"][:5]
    prepare_orders(bids, price, 1)
    prepare_orders(asks, price, -1)


    if last_price != 0:
        list.extend([price/last_price])
    else:
        list.extend([0])
        
    
    if last_time != 0:
        list.extend([int(state['timestamp'])/last_time])
    else:
        list.extend([0])
        
    last_time = int(state['timestamp'])
    
    y = dt.get_from_index(dt.index + steps)["price"]
    
    if y > price:
        y = onehot_encoded(0)
    elif y < price: 
        y = onehot_encoded(2)
    else:
        y = onehot_encoded(1)
        
    #datetime.fromtimestamp(int(state['timestamp']))
    
    return [list, y]


In [5]:
trainX = []
trainY = []
while len(trainX) < 30000: #(dt.max_steps() - steps*3):
    state = dt.next()
    state = get_state(state)
    trainX.append(state[0])
    trainY.append(state[1])
trainX = np.array(trainX)
trainY = np.array(trainY)
print(trainX.shape)
print(trainY.shape)

(30000, 12)
(30000, 3)


In [6]:
def get_batches(data, features, batch_size, seq_length):
    int_text = np.array(data)
    
    n_batches = int(len(data) / (batch_size * seq_length))

    xdata = np.array(data[: n_batches * batch_size * seq_length])
    
    data = xdata.reshape(-1, seq_length, features)
    
    #step1 = xdata.reshape(batch_size, -1)
    
    #print (step1.shape)
    #data = np.split(step1, n_batches, 1)
    
    return np.array(list(data))

In [7]:
print(trainX.shape)
batch_size = 1000
seq_length = 10
xdata = get_batches(trainX, trainX.shape[1], batch_size, seq_length)
print(xdata.shape)

(30000, 12)
(3000, 10, 12)


In [8]:
print(trainY.shape)
ydata = get_batches(trainY, trainY.shape[1], batch_size, seq_length)
#ydata = ydata.reshape(9900, seq_length, 3)
print(ydata.shape)

(30000, 3)
(3000, 10, 3)


In [9]:
y_train_reshaped = []
for i in range(ydata.shape[0]):
    y_train_reshaped.append(ydata[i, -1])

In [10]:
y_train_reshaped = np.array(y_train_reshaped)
print(y_train_reshaped.shape)

(3000, 3)


In [ ]:
nn = NeuralNetwork(0.00001, xdata.shape[2], y_train_reshaped.shape[1], xdata.shape[1])

In [ ]:
nn.train(xdata, y_train_reshaped)

Train on 2100 samples, validate on 900 samples
Epoch 1/150
2100/2100 [==============================] - 3s 1ms/step - loss: 0.7928 - mean_absolute_error: 0.3486 - acc: 0.5086 - val_loss: 0.7096 - val_mean_absolute_error: 0.3275 - val_acc: 0.6400
Epoch 2/150
2100/2100 [==============================] - 1s 674us/step - loss: 0.7705 - mean_absolute_error: 0.3444 - acc: 0.5038 - val_loss: 0.7358 - val_mean_absolute_error: 0.3402 - val_acc: 0.6400
Epoch 3/150
2100/2100 [==============================] - 1s 666us/step - loss: 0.7650 - mean_absolute_error: 0.3431 - acc: 0.5076 - val_loss: 0.7202 - val_mean_absolute_error: 0.3336 - val_acc: 0.6400
Epoch 4/150
2100/2100 [==============================] - 1s 658us/step - loss: 0.7657 - mean_absolute_error: 0.3421 - acc: 0.5176 - val_loss: 0.7439 - val_mean_absolute_error: 0.3445 - val_acc: 0.6400
Epoch 5/150
2100/2100 [==============================] - 1s 667us/step - loss: 0.7610 - mean_absolute_error: 0.3438 - acc: 0.5043 - val_loss: 0.7173 - 

In [ ]:
dt.index